# Задание 4

Рассматриваемый технологический узел представлен на рисунке ниже:

![Figure for task](.\fig_task_4.png "Схема узла")

Приборы показывают, что в узел поступает 90. По левой трубе уходит 35, а по правой - 60. 

*Допущения:*
 - физическими свойствами транспортируемой среды пренебрегаем;
 - в узле нет ни потерь, ни источников;
 - процесс установившейся.

---

## Решение

1. Для решения поставленной задачи применим методы DRGED - Data Reconciliation and Gross Error Detection (согласование данных и обнаружение грубых ошибок)

Согласование данных — это оценка переменных технологического процесса на основе информации, содержащейся в измерениях и моделях процесса. Модели процессов, используемые при согласовании данных, обычно представляют собой уравнения сохранения массы и энергии.

Основное применение этих методов заключается в подстройке некорретных измерений с помощью технологических инструментов (в том числе с использованием программных комплексов, таких как *IDAES Toolkit*), а также в обнаружении проблем и отказов в измерительном оборудовании.

